# 进行质量平衡分析

In [ ]:
import pandas as pd
from cobra.io import load_json_model,read_sbml_model

def check_mass_balance(sbml_file):
    """
    Detect chemical mass balance in a metabolic model (excluding boundary reactions).

    Args:
        sbml_file (str): Path to the SBML/JSON file.

    Returns:
        unbalanced_reactions (list): List of unbalanced reactions with details.
    """
    # Load the model
    model = read_sbml_model(sbml_file)

    # Store unbalanced reactions information
    unbalanced_reactions = []

    # Iterate through all reactions
    for reaction in model.reactions:
        # Skip boundary reactions (starting with EX_)
        if reaction.id.startswith("EX_"):
            continue

        # Get all metabolites and their stoichiometric coefficients
        metabolites = reaction.metabolites

        # Initialize element balance dictionary
        element_balance = {}

        for metabolite, coeff in metabolites.items():
            # Skip metabolites with no chemical formula
            if metabolite.formula is None:
                continue

            # Skip metabolites with invalid formula (elements is None)
            if metabolite.elements is None:
                continue

            # Analyze the chemical formula to break it into elements and counts
            for element, count in metabolite.elements.items():
                # Update the balance dictionary
                element_balance[element] = element_balance.get(element, 0) + count * coeff

        # Check if all elements are balanced
        unbalanced_elements = {e: v for e, v in element_balance.items() if abs(v) > 1e-6}
        if unbalanced_elements:
            # If there are unbalanced elements, record the reaction
            unbalanced_reactions.append({
                "reaction_id": reaction.id,
                "reaction_name": reaction.name,
                "unbalanced_elements": unbalanced_elements
            })

    return unbalanced_reactions

def export_to_excel(unbalanced_reactions, output_file):
    """
    Export unbalanced reaction results to an Excel file.
    """
    data = []
    for reaction in unbalanced_reactions:
        data.append({
            "Reaction ID": reaction["reaction_id"],
            "Reaction Name": reaction["reaction_name"],
            "Unbalanced Elements": ", ".join(f"{k}: {v}" for k, v in reaction["unbalanced_elements"].items())
        })

    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)
    print(f"Results exported to {output_file}")

# Main function
if __name__ == "__main__":
    sbml_file_path = "iCZ871_CGXII.xml"
    output_file_path = "iCZ871_CGXII_massunbanlancerxn.xlsx"

    unbalanced = check_mass_balance(sbml_file_path)

    if not unbalanced:
        print("All reactions are chemically balanced!")
    else:
        print(f"Found {len(unbalanced)} unbalanced reactions, exporting to file...")
        export_to_excel(unbalanced, output_file_path)

Set parameter Username
Set parameter LicenseID to value 2723056
Academic license - for non-commercial use only - expires 2026-10-16


In [1]:
import cobra
import memote
import memote.support.consistency as consistency

model = cobra.io.read_sbml_model("iCZ871_CGXII.xml")
is_consistent = consistency.check_stoichiometric_consistency(model)

print(f"模型的化学计量一致性: {is_consistent}")

if not is_consistent:
    # 如果模型不一致,使用 find_unconserved_metabolites 函数获取不一致的代谢物列表
    unconserved_mets = consistency.find_unconserved_metabolites(model)
    print("Inconsistent metabolites:")
    print(unconserved_mets)
else:
    print("模型是化学计量一致的,所有代谢物都满足质量守恒.")

Set parameter Username
Set parameter LicenseID to value 2723056
Academic license - for non-commercial use only - expires 2026-10-16


No objective coefficients in model. Unclear what should be optimized


模型的化学计量一致性: True
模型是化学计量一致的,所有代谢物都满足质量守恒.


In [2]:
def find_periplasm_metabolites(model):
    # 获取所有代谢物
    mets = model.metabolites
    
    # 筛选带有"_p"后缀的代谢物
    periplasm_mets = []
    for met in mets:
        if met.id.endswith('_p0'):
            periplasm_mets.append(met.id)
    
    # 排序并打印结果
    periplasm_mets.sort()
    print(f"Found {len(periplasm_mets)} periplasmic metabolites:")
    for met in periplasm_mets:
        print(met)
    
    return periplasm_mets

model = cobra.io.read_sbml_model("other model/iZM547.xml")
find_periplasm_metabolites(model)

CobraSBMLError: Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `cobra.io.sbml.validate_sbml_model` function or via the online validator at https://sbml.org/validator_servlet/ .
	`(model, errors) = validate_sbml_model(filename)`
If the model is valid and cannot be read please open an issue at https://github.com/opencobra/cobrapy/issues .